# Southeast Asian Android Phone Marketplace Analysis
## AI-Powered Analytics Dashboard

This notebook analyzes e-commerce data from a Southeast Asian Android phone marketplace using AI agents.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from openai import OpenAI
from swarm import Swarm, Agent
import warnings
warnings.filterwarnings('ignore')

### Load and Preprocess Data

In [ ]:
# Load the dataset
df = pd.read_csv('ai first sales data - sales.csv')

# Data preprocessing function
def preprocess_data(df):
    df_clean = df.copy()
    
    # Convert numeric columns
    numeric_columns = ['revenue', 'transactions', 'pageviews', 'visits', 
                      'productClick', 'addToCart', 'checkout', 'ad_spend']
    
    for col in numeric_columns:
        if col in df_clean.columns:
            if df_clean[col].dtype == 'object':
                df_clean[col] = df_clean[col].str.replace('₱', '').str.replace(',', '')
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
    
    # Convert date column
    if 'date' in df_clean.columns:
        df_clean['date'] = pd.to_datetime(df_clean['date'])
    
    # Fill missing values
    df_clean[numeric_columns] = df_clean[numeric_columns].fillna(0)
    
    return df_clean

# Preprocess the data
df_clean = preprocess_data(df)
print(f"Dataset loaded successfully with {len(df_clean)} rows")
df_clean.head()

### Data Quality Analysis

In [ ]:
def analyze_data_quality(df):
    # Missing values analysis
    missing_values = df.isnull().sum()
    missing_percentage = (missing_values / len(df)) * 100
    
    # Basic statistics
    numeric_stats = df.describe()
    
    # Data types
    dtypes = df.dtypes
    
    return missing_percentage, numeric_stats, dtypes

missing_pct, stats, dtypes = analyze_data_quality(df_clean)
print("\nMissing Values (%):\n", missing_pct)
print("\nData Types:\n", dtypes)
print("\nNumeric Statistics:\n", stats)

### Marketing Channel Analysis

In [ ]:
# Analyze channel performance
channel_metrics = df_clean.groupby('source').agg({
    'revenue': 'sum',
    'transactions': 'sum',
    'ad_spend': 'sum'
}).reset_index()

# Calculate ROI
channel_metrics['ROI'] = (channel_metrics['revenue'] - channel_metrics['ad_spend']) / channel_metrics['ad_spend']

# Visualize channel performance
fig = px.bar(channel_metrics, x='source', y='revenue',
             title='Revenue by Marketing Channel')
fig.show()

# ROI visualization
fig = px.bar(channel_metrics, x='source', y='ROI',
             title='ROI by Marketing Channel')
fig.show()

channel_metrics

### Customer Journey Analysis

In [ ]:
# Create funnel data
funnel_data = {
    'Stage': ['Pageviews', 'Product Clicks', 'Add to Cart', 'Checkout', 'Transactions'],
    'Count': [
        df_clean['pageviews'].sum(),
        df_clean['productClick'].sum(),
        df_clean['addToCart'].sum(),
        df_clean['checkout'].sum(),
        df_clean['transactions'].sum()
    ]
}

# Create funnel visualization
fig = go.Figure(go.Funnel(y=funnel_data['Stage'], x=funnel_data['Count']))
fig.update_layout(title='Customer Journey Funnel')
fig.show()

# Calculate conversion rates
conversion_rates = pd.DataFrame({
    'Stage': ['Click-through Rate', 'Add-to-Cart Rate', 'Checkout Rate', 'Purchase Rate'],
    'Rate': [
        df_clean['productClick'].sum() / df_clean['pageviews'].sum() * 100,
        df_clean['addToCart'].sum() / df_clean['productClick'].sum() * 100,
        df_clean['checkout'].sum() / df_clean['addToCart'].sum() * 100,
        df_clean['transactions'].sum() / df_clean['checkout'].sum() * 100
    ]
})

conversion_rates

### Revenue Analysis

In [ ]:
# Daily revenue trend
daily_revenue = df_clean.groupby('date')['revenue'].sum().reset_index()

# Plot revenue trend
fig = px.line(daily_revenue, x='date', y='revenue',
              title='Daily Revenue Trend')
fig.show()

# Revenue statistics
revenue_stats = daily_revenue['revenue'].describe()
print("\nRevenue Statistics:")
print(revenue_stats)

# Calculate month-over-month growth
monthly_revenue = df_clean.groupby(df_clean['date'].dt.to_period('M'))['revenue'].sum()
monthly_growth = monthly_revenue.pct_change() * 100

print("\nMonth-over-Month Growth:")
print(monthly_growth)

### Device Analysis

In [ ]:
# Device performance metrics
device_metrics = df_clean.groupby('device').agg({
    'revenue': 'sum',
    'transactions': 'sum',
    'pageviews': 'sum'
}).reset_index()

# Calculate conversion rate by device
device_metrics['conversion_rate'] = (device_metrics['transactions'] / device_metrics['pageviews']) * 100

# Visualize device performance
fig = px.pie(device_metrics, values='revenue', names='device',
             title='Revenue Distribution by Device')
fig.show()

device_metrics

### Key Insights and Recommendations

In [ ]:
# Generate insights based on the analysis
print("Key Insights:")
print("1. Channel Performance:")
top_channel = channel_metrics.sort_values('revenue', ascending=False).iloc[0]
print(f"   - Top performing channel: {top_channel['source']} with ₱{top_channel['revenue']:,.2f} in revenue")

print("\n2. Customer Journey:")
print(f"   - Overall conversion rate: {(df_clean['transactions'].sum() / df_clean['pageviews'].sum() * 100):.2f}%")

print("\n3. Device Usage:")
top_device = device_metrics.sort_values('revenue', ascending=False).iloc[0]
print(f"   - Preferred device: {top_device['device']} with {top_device['conversion_rate']:.2f}% conversion rate")

print("\nRecommendations:")
print("1. Marketing Strategy:")
print(f"   - Focus budget allocation on {top_channel['source']} channel")
print("   - Optimize campaigns for better ROI")

print("\n2. User Experience:")
print("   - Improve checkout process to reduce abandonment")
print(f"   - Optimize for {top_device['device']} experience")

print("\n3. Revenue Growth:")
print("   - Implement personalized marketing campaigns")
print("   - Focus on high-value customer segments")